# Jupyter Notebook for inter tissue WGCNA

## Import libraries

In [1]:
library(WGCNA)
options(stringsAsFactors = FALSE)
suppressMessages(WGCNA::allowWGCNAThreads())

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: 'fastcluster'


The following object is masked from 'package:stats':

    hclust





Attaching package: 'WGCNA'


The following object is masked from 'package:stats':

    cor




Allowing multi-threading with up to 8 threads.


## Helper functions

In [ ]:
.extract_donor <- function(x) sub("^([^-]+-[^-]+).*", "\\1", x)

.aggregate_by_donor <- function(mat) {
    d <- .extract_donor(rownames(mat))
    split_idx <- split(seq_len(nrow(mat)), d)
    res <- do.call(rbind, lapply(split_idx, function(ix) {
        colMeans(mat[ix, , drop = FALSE], na.rm = TRUE)
    }))
    rownames(res) <- names(split_idx)
    storage.mode(res) <- "double"
    res
}
split_gene_id <- function(x) {
  x <- as.character(x)
  tissue <- sub("_([^_]*)$", "", x)
  gene   <- sub("^.*_", "", x) 
  list(tissue = tissue, gene = gene)
}
.make_CT_map <- function(tissues, beta) {
  if (length(tissues) < 2) return(setNames(numeric(0), character(0)))
  pairs <- t(combn(tissues, 2))
  keys  <- paste(pairs[,1], pairs[,2], sep = "||")
  setNames(rep.int(beta, length(keys)), keys)
}